In [1]:
%load_ext autoreload
%autoreload 2 
%matplotlib tk

In [ ]:
import pianoq
from pianoq.lab.asi_cam import ASICam
from TC300_COMMAND_LIB import TC300

In [3]:
tc = TC300()
serial = tc.list_devices()[0][0]
tc.open(serial, 115200, 3)
cam = ASICam()

In [ ]:
base_path = r'G:\My Drive\People\Ronen\PHD\SPDC2025\phase_matching'
cam.set_exposure(1)
tc.set_target_temperature(2, 75)

0

In [21]:
import os 
import time 

t = 0
while t < 72:
    time.sleep(2)
    tt = [0]
    tc.get_actual_temperature(2, tt)
    t = round(tt[0], 4)
    cam.save_image(os.path.join(base_path, f"T={t}.fits"))

In [ ]:
tc.close()
cam.close()

0

In [2]:
from pianoq.lab.photon_counter import PhotonCounter

can't import TimeTagger
could not import ThorlabsRotatingServoMotor


In [3]:
ph = PhotonCounter(serial_port='COM8')

In [ ]:
while True:
    x = ph.read()
    print(x)

In [18]:
x

(array([0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0.]),
 1.0)